In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import csv
import random
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostRegressor
random.seed(1)

In [ ]:
# Read The data
training_set = pd.read_json('./train_set.json')
test_set = pd.read_json('./test_set.json')

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(training_set, test_size=0.25, random_state=42)

In [4]:
X_train

,document,summary,label
3836,Former QPR manager Harry Redknapp says that To...,Harry Redknapp says Tottenham have not pulled ...,1
6408,Franck Ribery has dismissed reports linking Wo...,kevin de bruyne has been linked with a move to...,0
4840,A man was left with a swollen and painful peni...,"Szilveszter , 24 , had petroleum jelly injecte...",1
527,Children who experience stressful events such ...,"Swedish scientists analysed more than 10,000 f...",1
6105,Atlanta (CNN)A fake name on a Facebook post ca...,Sheriff 's spokeswoman : Ebony Dickens is out ...,1
...,...,...,...
5226,Straight after Easter on planet football comes...,Chelsea boss Jose Mourinho has spoken in defen...,1
5390,A 12-year-old sports fan from Texas - who's be...,Nicholas Connors gave a rousing performance ah...,0
860,This was tricky for Steve Cotterill. He did no...,Steve Cotterill has taken Bristol City to the ...,0
7603,John Carver has challenged his Newcastle playe...,John Carver says his players must lift the pre...,1


Doc2Vec Features

In [5]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\ALike\.conda\envs\tensorflow\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:
import gensim
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def read_corpus(df, tokens_only=False):
    for idx, row in df.iterrows():
        tokens = gensim.utils.simple_preprocess(row['summary'])
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [idx])

In [7]:
train_corpus = list(read_corpus(X_train))
test_corpus = list(read_corpus(X_test, tokens_only=True))

In [25]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=500, min_count=2, epochs=1000)
model.build_vocab(train_corpus)

In [26]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
train_lst_vecs = []
test_lst_vecs = []
for i in enumerate(train_corpus):
    train_lst_vecs.append(model.infer_vector(i[1][0]))
for i in test_corpus:
    test_lst_vecs.append(model.infer_vector(i))
X_train['doc2vec'] = train_lst_vecs
X_test['doc2vec'] = test_lst_vecs

In [ ]:
train_X = np.stack(train_lst_vecs, axis=0)
test_X = np.stack(test_lst_vecs, axis=0)

In [ ]:
train_X.shape

In [ ]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(train_X, X_train['label'])
RF_predictions = rf.predict(test_X)
# Use score method to get accuracy of model
acc_RF = accuracy_score(X_test['label'], RF_predictions)
print(acc_RF)

score_RF = rf.score(train_X, X_train['label'])
print(score_RF)

In [ ]:
acc_RF = accuracy_score(X_test['label'], RF_predictions)
print(acc_RF)

In [ ]:
xgbc = XGBClassifier()
xgbc.fit(train_X, X_train['label'])
xgbc_predictions = xgbc.predict(test_X)
print(accuracy_score(xgbc_predictions, X_test['label']))
# Use score method to get accuracy of model
score_RF = xgbc.score(np.array(train_X), X_train['label'])
print(score_RF)

In [ ]:
# Write predictions to a file
with open("./results/submission_xgboost_doc2vec_200_20thmarch.csv", "w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','label'])
    for i, row in enumerate(RF_predictions):
        csv_out.writerow([i, row])

In [ ]:
np.unique(xgbc_predictions, return_counts=True)

In [ ]:
np.unique(RF_predictions, return_counts=True)

In [ ]:
X_test.shape